In [37]:
import pandas as pd

In [38]:
#feature engineering
df = pd.read_csv("../preprocessing/train_data.csv")
#time-based features
# Define mapping of month names to numbers
month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

df['Month'] = df['Month'].map(month_mapping)

# Sort by Year and Month (ascending order)
df = df.sort_values(by=['Year', 'Month'], ascending=[True, True]).reset_index(drop=True)

# convert month to integer type
df['Month'] = df['Month'].astype(int)

# Create a 'Quarter' feature
df['Quarter'] = ((df['Month'] - 1) // 3) + 1

df['Years_Since_2000'] = df['Year'] - 2000

df['Departures_Growth'] = df.groupby('Country')['Departures'].pct_change()
df['Arrivals_Growth'] = df.groupby('Country')['Arrivals'].pct_change()


In [39]:
#geographical data
# Define a dictionary to map countries to broader geographical regions
region_mapping = {
    'France': 'Europe',
    'Germany': 'Europe',
    'United Kingdom': 'Europe',
    
    'Indonesia': 'South East Asia',
    'Malaysia': 'South East Asia',
    'Philippines': 'South East Asia',
    'Thailand': 'South East Asia',
    'Vietnam': 'South East Asia',

    'Mainland China': 'East Asia',
    'Japan': 'East Asia',
    'Hong Kong': 'East Asia',
    
    'South Asia': 'South Asia',  # Already a region
    'South East Asia': 'South East Asia',  # Already a region
    'North East Asia': 'East Asia',  # Adjusted for consistency

    'Middle East': 'Middle East',  # Already a region
    'North America': 'North America',  # Already a region
    'Oceania': 'Oceania',  # Already a region
    'Europe': 'Europe',  # Already a region
}

# Standardize country names (remove extra spaces)
df['Country'] = df['Country'].str.strip()

# Create a new column 'Region' based on the mapping
df['Region'] = df['Country'].map(region_mapping)

# Check if any country was not mapped (helps in debugging)
unmapped_countries = df[df['Region'].isna()]['Country'].unique()
print("Unmapped Countries:", unmapped_countries)


Unmapped Countries: []


In [40]:
#one-hot encode region
df = pd.get_dummies(df, columns=['Region'], prefix='Region')

In [41]:
# Fill NaN values in the first few rows (likely due to pct_change()) with 0
df['Departures_Growth'] = df.groupby('Country')['Departures_Growth'].fillna(0)
df['Arrivals_Growth'] = df.groupby('Country')['Arrivals_Growth'].fillna(0)


C:\Users\Vianiece\AppData\Local\Temp\ipykernel_4580\3205859310.py:2: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['Departures_Growth'] = df.groupby('Country')['Departures_Growth'].fillna(0)
C:\Users\Vianiece\AppData\Local\Temp\ipykernel_4580\3205859310.py:3: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['Arrivals_Growth'] = df.groupby('Country')['Arrivals_Growth'].fillna(0)


In [42]:
# Save the updated dataset back to the train CSV
df.to_csv("../preprocessing/train_data.csv", index=False)

print("Updated train_data.csv saved successfully!")


Updated train_data.csv saved successfully!


In [ ]:
import pandas as pd

# Load the holiday data
holidays_df = pd.read_csv("../Dataset/singapore_holidays.csv")

# Convert 'Date' to datetime, handling the format 'YYYY-MM-DD'
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'], format='%Y-%m-%d')

# Extract 'Month' and 'Year' from 'Date' for matching with the main dataset
holidays_df['Month'] = holidays_df['Date'].dt.month
holidays_df['Year'] = holidays_df['Date'].dt.year

# Load the main dataset
df = pd.read_csv("../preprocessing/train_data.csv")

# Merge the main dataset with holiday dataset on both Year and Month
df = pd.merge(df, holidays_df[['Year', 'Month', 'Event']], 
              how='left', on=['Year', 'Month'])

print(df.head())

# Create a flag for holiday presence in the month
df['Is_Holiday_Month'] = df['Event'].notna().astype(int)

# Optionally, drop the 'Event' column if you just want the flag
df = df.drop(columns=['Event'])

# Save the updated dataset with holidays
df.to_csv("train_data_with_holidays.csv", index=False)


   Month  Year         Country  Departures  Arrivals  Quarter  \
0      1  2000   North America     29012.0   26225.0        1   
1      1  2000   North America     29012.0   26225.0        1   
2      1  2000  United Kingdom     46677.0   45630.0        1   
3      1  2000  United Kingdom     46677.0   45630.0        1   
4      1  2000         Germany     23172.0   24251.0        1   

   Years_Since_2000  Departures_Growth  Arrivals_Growth  Region_East Asia  \
0                 0                0.0              0.0             False   
1                 0                0.0              0.0             False   
2                 0                0.0              0.0             False   
3                 0                0.0              0.0             False   
4                 0                0.0              0.0             False   

   Region_Europe  Region_Middle East  Region_North America  Region_Oceania  \
0          False               False                  True          

In [46]:
# Check for duplicates in the entire dataset
duplicates = df[df.duplicated()]

# Print the duplicates
print(f"Number of duplicate rows: {duplicates.shape[0]}")
print(duplicates)

# If you want to drop the duplicates, you can use:
train_df = df.drop_duplicates()

# Save the dataset after removing duplicates
train_df.to_csv("train_data_with_holidays.csv", index=False)

# Check the updated shape
print(f"Updated Training set size after removing duplicates: {train_df.shape}")


Number of duplicate rows: 1296
      Month  Year         Country  Departures  Arrivals  Quarter  \
1         1  2000   North America     29012.0   26225.0        1   
3         1  2000  United Kingdom     46677.0   45630.0        1   
5         1  2000         Germany     23172.0   24251.0        1   
7         1  2000          France     11098.0   11469.0        1   
9         1  2000          Europe    118255.0  119735.0        1   
...     ...   ...             ...         ...       ...      ...   
4889      8  2019     Middle East     91493.0   79405.0        3   
4891      8  2019      South Asia    244304.0  262089.0        3   
4892      8  2019      South Asia    244304.0  262089.0        3   
4894      8  2019           Japan    151982.0  149519.0        3   
4895      8  2019           Japan    151982.0  149519.0        3   

      Years_Since_2000  Departures_Growth  Arrivals_Growth  Region_East Asia  \
1                    0           0.000000          0.00000             F